First thing to do is to request the Wikipedia page

In [1]:
import requests

wiki_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = requests.get(wiki_link)

I used the BeautifulSoup package to get the entire HTML text

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(wiki_page.text, 'html.parser')

Then I used the find_all function to find all table tag and take only the first one that was the one I was interested to

In [3]:
table = soup.find_all('table')[0]

And finally I used again the find_all function to get the rows of the table as a list

In [4]:
rows = table.find_all('tr')

To create the dataframe:

In [5]:
import pandas as pd

column_names = ['PostalCode','Borough','Neighborhood']
pc_can = pd.DataFrame(columns=column_names)

Here I used again the find_all function to retrive all the elements in a row. Then I took only the text between the tag and assigned to each column in the dataframe:

In [6]:
for row in rows[1:]:
    elements = row.find_all('td')
    pc = elements[0].text
    borough = elements[1].text
    neighbh = elements[2].text
    if borough!="Not assigned":
        pc_can = pc_can.append({'PostalCode': pc, 'Borough': borough, 'Neighborhood': neighbh[:-1]}, ignore_index=True)    

In [7]:
pc_can.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


To merge all the rows with the same PostalCode I used the groupby function on that particular column and merged the other column with the aggregate function with two methods: for Borough column it will take the first value of the merged rows, for the Neighborhood one it will join the values of the merged rows with a coma between the two values.

In [8]:
pc_can_new = pc_can.groupby('PostalCode', as_index=False).agg({'Borough':'first','Neighborhood':', '.join})
pc_can_new.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
pc_can_new.shape

(103, 3)

In [18]:
df = pd.read_csv('Geospatial_Coordinates.csv')
df.rename({"Postal Code":"PostalCode"}, axis='columns', inplace=True)

final = pd.merge(pc_can_new, df, on="PostalCode")

final.head()
    

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
